In [1]:
# problem 3
n = 3
a = 7
x = 1

# list all possible y we can get with measurement
ys = []
for y in range(2 ** n):
    t = 0
    for i in range(n):
        t ^= (((y & a) >> i) & 1)
    if t == 0:
        ys.append(y)
print(ys)

# compute number of independent vectors
def nindependent(vs):
    b = [0] * n # basis
    answer = 0
    for v in vs:
        x = v
        for i in range(n):
            if ((x >> i) & 1) == 1:
                if b[i] == 0:
                    b[i] = x
                    answer += 1
                    break
                else:
                    x ^= b[i]
    return answer

# choosing n + x = 4 random vectors
total = len(ys) ** 4
good = 0
for y1 in ys:
    for y2 in ys:
        for y3 in ys:
            for y4 in ys:
                if nindependent([y1, y2, y3, y4]) == 2:
                    good += 1
print(good / total)

# computing by formula
q = 1
for i in range(2, n + 1):
    q *= (1 - 1 / (2 ** (i + x)))
print(q)

[0, 3, 5, 6]
0.8203125
0.8203125


In [2]:
# problem 4
from qiskit import *
IBMQ.load_account()
simulator = BasicAer.get_backend('qasm_simulator')
circuit3 = QuantumCircuit(2, 2)
# Grover's search consists of four parts:
# 1. preparation of initial states using H^n
# 2. oracle, which is also V operator, which flips the sign of 0-th qbit in case of a match
# 3. W operator, with 2 qbits it is H_0 H_1 X_0 X_1 cZ_01 X_0 X_1 H_0 H_1
# 2 and 3 are repeated ~(pi/4)2^(n/2) times, in case of n=2 -- only once
# 4. measurement
# part 1: preparation
circuit3.h(0)
circuit3.h(1)
# part 2: oracle for a = 3 (both bits are 1) is cZ_10 = H_0 cX_10 H_0
circuit3.h(0)
circuit3.cx(1, 0)
circuit3.h(0)
# part 3: W operator = H_0 H_1 X_0 X_1 H_0 cX_10 H_0 X_0 X_1 H_0 H_1
circuit3.h(0)
circuit3.h(1)
circuit3.x(0)
circuit3.x(1)
circuit3.h(0)
circuit3.cx(1, 0)
circuit3.h(0)
circuit3.x(0)
circuit3.x(1)
circuit3.h(0)
circuit3.h(1)
# part 4: measurement
circuit3.measure([0, 1], [0, 1])
circuit3.draw()

/usr/local/lib/python3.7/dist-packages/qiskit/__init__.py:65: RuntimeWarning: Could not import the Aer provider from the qiskit-aer package. Install qiskit-aer or check your installation.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤M├
     ├───┤└───┘└─┬─┘├───┤├───┤└───┘└───┘└─┬─┘├───┤├───┤└┬─┬┘└╥┘
q_1: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─┤M├──╫─
     └───┘          └───┘└───┘               └───┘└───┘ └╥┘  ║ 
c: 2/════════════════════════════════════════════════════╩═══╩═
                                                         1   0

In [3]:
job = execute(circuit3, simulator)
result = job.result()
counts = result.get_counts(circuit3)
counts

{'11': 1024}

In [4]:
# circuit for a = 0
circuit0 = QuantumCircuit(2, 2)
# part 1: preparation
circuit0.h(0)
circuit0.h(1)
# part 2: oracle for a = 0 (both bits are 0) is X_0 X_1 cZ_10 X_0 X_1 = X_0 X_1 H_0 cX_10 H_0 X_0 X_1 
circuit0.x(0)
circuit0.x(1)
circuit0.h(0)
circuit0.cx(1, 0)
circuit0.h(0)
circuit0.x(0)
circuit0.x(1)
# part 3: W operator = H_0 H_1 X_0 X_1 H_0 cX_10 H_0 X_0 X_1 H_0 H_1
circuit0.h(0)
circuit0.h(1)
circuit0.x(0)
circuit0.x(1)
circuit0.h(0)
circuit0.cx(1, 0)
circuit0.h(0)
circuit0.x(0)
circuit0.x(1)
circuit0.h(0)
circuit0.h(1)
# part 4: measurement
circuit0.measure([0, 1], [0, 1])
circuit0.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤M├
     ├───┤├───┤└───┘└─┬─┘├───┤├───┤├───┤└───┘└───┘└─┬─┘├───┤├───┤└┬─┬┘└╥┘
q_1: ┤ H ├┤ X ├───────■──┤ X ├┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─┤M├──╫─
     └───┘└───┘          └───┘└───┘└───┘               └───┘└───┘ └╥┘  ║ 
c: 2/══════════════════════════════════════════════════════════════╩═══╩═
                                                                   1   0

In [5]:
# circuit for a = 1
circuit1 = QuantumCircuit(2, 2)
# part 1: preparation
circuit1.h(0)
circuit1.h(1)
# part 2: oracle for a = 1 (bit 0 is 1, bit 1 is 0) is X_1 cZ_10 X_1 = X_1 H_0 cX_10 H_0 X_1 
circuit1.x(1)
circuit1.h(0)
circuit1.cx(1, 0)
circuit1.h(0)
circuit1.x(1)
# part 3: W operator = H_0 H_1 X_0 X_1 H_0 cX_10 H_0 X_0 X_1 H_0 H_1
circuit1.h(0)
circuit1.h(1)
circuit1.x(0)
circuit1.x(1)
circuit1.h(0)
circuit1.cx(1, 0)
circuit1.h(0)
circuit1.x(0)
circuit1.x(1)
circuit1.h(0)
circuit1.h(1)
# part 4: measurement
circuit1.measure([0, 1], [0, 1])
circuit1.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤M├
     ├───┤├───┤└─┬─┘├───┤├───┤├───┤└───┘└─┬─┘├───┤├───┤└┬─┬┘└╥┘
q_1: ┤ H ├┤ X ├──■──┤ X ├┤ H ├┤ X ├───────■──┤ X ├┤ H ├─┤M├──╫─
     └───┘└───┘     └───┘└───┘└───┘          └───┘└───┘ └╥┘  ║ 
c: 2/════════════════════════════════════════════════════╩═══╩═
                                                         1   0

In [6]:
# circuit for a = 2
circuit2 = QuantumCircuit(2, 2)
# part 1: preparation
circuit2.h(0)
circuit2.h(1)
# part 2: oracle for a = 2 (bit 0 is 0, bit 1 is 1) is X_0 cZ_10 X_0 = X_0 H_0 cX_10 H_0 X_0 
circuit2.x(0)
circuit2.h(0)
circuit2.cx(1, 0)
circuit2.h(0)
circuit2.x(0)
# part 3: W operator = H_0 H_1 X_0 X_1 H_0 cX_10 H_0 X_0 X_1 H_0 H_1
circuit2.h(0)
circuit2.h(1)
circuit2.x(0)
circuit2.x(1)
circuit2.h(0)
circuit2.cx(1, 0)
circuit2.h(0)
circuit2.x(0)
circuit2.x(1)
circuit2.h(0)
circuit2.h(1)
# part 4: measurement
circuit2.measure([0, 1], [0, 1])
circuit2.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┐
q_0: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤M├
     ├───┤└───┘└───┘└─┬─┘├───┤├───┤└───┘└───┘└───┘└─┬─┘├───┤├───┤└┬─┬┘└╥┘
q_1: ┤ H ├────────────■──┤ H ├┤ X ├─────────────────■──┤ X ├┤ H ├─┤M├──╫─
     └───┘               └───┘└───┘                    └───┘└───┘ └╥┘  ║ 
c: 2/══════════════════════════════════════════════════════════════╩═══╩═
                                                                   1   0

In [7]:
job = execute(circuit0, simulator)
result = job.result()
counts = result.get_counts(circuit0)
counts

{'00': 1024}

In [8]:
job = execute(circuit1, simulator)
result = job.result()
counts = result.get_counts(circuit1)
counts

{'01': 1024}

In [9]:
job = execute(circuit2, simulator)
result = job.result()
counts = result.get_counts(circuit2)
counts

{'10': 1024}

In [10]:
# if VW is repeated more times than needed, then in the plane (a, phi) we get more rotation than needed, and get wrong result,
# let's test
# part 1: preparation
circuit5 = QuantumCircuit(2, 2)
circuit5.h(0)
circuit5.h(1)
# part 2: oracle for a = 3 (both bits are 1) is cZ_10 = H_0 cX_10 H_0
circuit5.h(0)
circuit5.cx(1, 0)
circuit5.h(0)
# part 3: W operator = H_0 H_1 X_0 X_1 H_0 cX_10 H_0 X_0 X_1 H_0 H_1
circuit5.h(0)
circuit5.h(1)
circuit5.x(0)
circuit5.x(1)
circuit5.h(0)
circuit5.cx(1, 0)
circuit5.h(0)
circuit5.x(0)
circuit5.x(1)
circuit5.h(0)
circuit5.h(1)
# parts 2 and 3 are repeated
circuit5.h(0)
circuit5.cx(1, 0)
circuit5.h(0)
# 
circuit5.h(0)
circuit5.h(1)
circuit5.x(0)
circuit5.x(1)
circuit5.h(0)
circuit5.cx(1, 0)
circuit5.h(0)
circuit5.x(0)
circuit5.x(1)
circuit5.h(0)
circuit5.h(1)
# part 4: measurement
circuit5.measure([0, 1], [0, 1])
circuit5.draw()

┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q_0: ┤ H ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ H ├┤ X ├┤ H ├»
     ├───┤└───┘└─┬─┘├───┤├───┤└───┘└───┘└─┬─┘├───┤├───┤└───┘└───┘└─┬─┘├───┤»
q_1: ┤ H ├───────■──┤ H ├┤ X ├────────────■──┤ X ├┤ H ├────────────■──┤ H ├»
     └───┘          └───┘└───┘               └───┘└───┘               └───┘»
c: 2/══════════════════════════════════════════════════════════════════════»
                                                                           »
«     ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌─┐
«q_0: ┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤M├
«     ├───┤└───┘└───┘└─┬─┘├───┤├───┤└┬─┬┘└╥┘
«q_1: ┤ X ├────────────■──┤ X ├┤ H ├─┤M├──╫─
«     └───┘               └───┘└───┘ └╥┘  ║ 
«c: 2/════════════════════════════════╩═══╩═
«                                     1   0

In [11]:
job = execute(circuit5, simulator)
result = job.result()
counts = result.get_counts(circuit5)
counts

{'01': 266, '11': 248, '00': 257, '10': 253}

In [ ]:
# result is wrong as expected